In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**1.**

In [ ]:
!pip install sklearn pandas

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import numpy as np
from collections import Counter


class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self.build_tree(X, y)

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def build_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # proveriti kriterijum stajanja
        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            leaf_value = self.getMCL(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        # find the best split
        best_feature, best_thresh = self.find_best(X, y, feat_idxs)

        # create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self.build_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self.build_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)


    def find_best(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:

                gain = self.getGains(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold


    def getGains(self, y, X_column, threshold):
        parent_entropy = self.getEntropy(y)

        # create children
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # izracunaj prosecnu (weighted) entropiju dece
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self.getEntropy(y[left_idxs]), self.getEntropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        information_gainz = parent_entropy - child_entropy
        return information_gainz

    def getEntropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p>0])

    # MOST COMMON LABEL
    def getMCL(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self.traverse(x, self.root) for x in X])

    def traverse(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self.traverse(x, node.left)
        return self.traverse(x, node.right)

def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test)

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

# Učitavanje podataka
crop_data = pd.read_csv('/content/drive/MyDrive/Colab/crop.csv')
crop_data.describe()

,Nitrogen,Phosphorus,Potassium,Temperature,Humidity,pH_Value,Rainfall
count,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000
mean,50.551818,53.362727,48.149091,25.616244,71.481779,6.469480,103.463655
std,36.917334,32.985883,50.647931,5.063749,22.263812,0.773938,54.958389
min,0.000000,5.000000,5.000000,8.825675,14.258040,3.504752,20.211267
25%,21.000000,28.000000,20.000000,22.769375,60.261953,5.971693,64.551686
50%,37.000000,51.000000,32.000000,25.598693,80.473146,6.425045,94.867624
75%,84.250000,68.000000,49.000000,28.561654,89.948771,6.923643,124.267508
max,140.000000,145.000000,205.000000,43.675493,99.981876,9.935091,298.560117


In [ ]:
crop_data['Crop'].value_counts()

Crop
Rice           100
Maize          100
Jute           100
Cotton         100
Coconut        100
Papaya         100
Orange         100
Apple          100
Muskmelon      100
Watermelon     100
Grapes         100
Mango          100
Banana         100
Pomegranate    100
Lentil         100
Blackgram      100
MungBean       100
MothBeans      100
PigeonPeas     100
KidneyBeans    100
ChickPea       100
Coffee         100
Name: count, dtype: int64

In [ ]:
crop_dict = {
    'Rice': 1,
    'Maize': 2,
    'Jute': 3,
    'Cotton': 4,
    'Coconut': 5,
    'Papaya': 6,
    'Orange': 7,
    'Apple': 8,
    'Muskmelon': 9,
    'Watermelon': 10,
    'Grapes': 11,
    'Mango': 12,
    'Banana': 13,
    'Pomegranate': 14,
    'Lentil': 15,
    'Blackgram': 16,
    'Mungbean': 17,
    'Mothbeans': 18,
    'Pigeonpeas': 19,
    'Kidneybeans': 20,
    'Chickpea': 21,
    'Coffee': 22
}

crop_data['Crop_Num']=crop_data['Crop'].map(crop_dict)
crop_data['Crop_Num'].value_counts()

Crop_Num
1.0     100
9.0     100
3.0     100
4.0     100
5.0     100
6.0     100
7.0     100
8.0     100
10.0    100
2.0     100
11.0    100
12.0    100
13.0    100
14.0    100
15.0    100
16.0    100
22.0    100
Name: count, dtype: int64

In [ ]:
crop_data.head()

,Nitrogen,Phosphorus,Potassium,Temperature,Humidity,pH_Value,Rainfall,Crop,Crop_Num
0,90,42,43,20.879744,82.002744,6.502985,202.935536,Rice,1.0
1,85,58,41,21.770462,80.319644,7.038096,226.655537,Rice,1.0
2,60,55,44,23.004459,82.320763,7.840207,263.964248,Rice,1.0
3,74,35,40,26.491096,80.158363,6.980401,242.864034,Rice,1.0
4,78,42,42,20.130175,81.604873,7.628473,262.717340,Rice,1.0


In [ ]:
from sklearn.model_selection import train_test_split

X = crop_data.drop(['Crop_Num', 'Crop'], axis=1)
y = crop_data['Crop_Num']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


X

,Nitrogen,Phosphorus,Potassium,Temperature,Humidity,pH_Value,Rainfall
0,90,42,43,20.879744,82.002744,6.502985,202.935536
1,85,58,41,21.770462,80.319644,7.038096,226.655537
2,60,55,44,23.004459,82.320763,7.840207,263.964248
3,74,35,40,26.491096,80.158363,6.980401,242.864034
4,78,42,42,20.130175,81.604873,7.628473,262.717340
...,...,...,...,...,...,...,...
2195,107,34,32,26.774637,66.413269,6.780064,177.774507
2196,99,15,27,27.417112,56.636362,6.086922,127.924610
2197,118,33,30,24.131797,67.225123,6.362608,173.322839
2198,117,32,34,26.272418,52.127394,6.758793,127.175293


In [ ]:
from sklearn.preprocessing import MinMaxScaler
ms = MinMaxScaler()

X_train = ms.fit_transform(X_train)
X_test = ms.transform(X_test)

X_train

array([[0.12142857, 0.07857143, 0.045     , ..., 0.9089898 , 0.48532225,
        0.29685161],
       [0.26428571, 0.52857143, 0.07      , ..., 0.64257946, 0.56594073,
        0.17630752],
       [0.05      , 0.48571429, 0.1       , ..., 0.57005802, 0.58835229,
        0.08931844],
       ...,
       [0.07857143, 0.22142857, 0.13      , ..., 0.43760347, 0.46198144,
        0.28719815],
       [0.07857143, 0.85      , 0.995     , ..., 0.76763665, 0.44420505,
        0.18346657],
       [0.22857143, 0.52142857, 0.085     , ..., 0.56099735, 0.54465022,
        0.11879596]])

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()


sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

X_train

array([[-9.03426596e-01, -1.12616170e+00, -6.68506601e-01, ...,
         9.36586183e-01,  1.93473784e-01,  5.14970176e-03],
       [-3.67051340e-01,  7.70358846e-01, -5.70589522e-01, ...,
        -1.00470485e-01,  8.63917548e-01, -6.05290566e-01],
       [-1.17161422e+00,  5.89737842e-01, -4.53089028e-01, ...,
        -3.82774991e-01,  1.05029771e+00, -1.04580687e+00],
       ...,
       [-1.06433917e+00, -5.24091685e-01, -3.35588533e-01, ...,
        -8.98381379e-01, -6.34357580e-04, -4.37358211e-02],
       [-1.06433917e+00,  2.12501638e+00,  3.05234239e+00, ...,
         3.86340190e-01, -1.48467347e-01, -5.69036842e-01],
       [-5.01145154e-01,  7.40255346e-01, -5.11839275e-01, ...,
        -4.18045489e-01,  6.86860180e-01, -8.96531475e-01]])

In [ ]:
y_train = y_train.fillna(0)

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)

Y_pred = decision_tree.predict(X_test)

acc_decision_tree = round(accuracy(y_test, Y_pred) * 100, 2)
print(f"Sklearn {round(acc_decision_tree,2,)}", "%")

Sklearn 73.41 %


In [ ]:
def recommendation(N,P,k,temperature,humidity,ph,rainfal):
    features = np.array([[N,P,k,temperature,humidity,ph,rainfal]])
    transformed_features = ms.fit_transform(features)
    transformed_features = sc.fit_transform(transformed_features)
    prediction = decision_tree.predict(transformed_features).reshape(1,-1)

    return prediction[0]

N = 40
P = 50
k = 50
temperature = 40.0
humidity = 20
ph = 100
rainfall = 100

crop_dict = {1: "Rice", 2: "Maize", 3: "Jute", 4: "Cotton", 5: "Coconut", 6: "Papaya", 7: "Orange",
                 8: "Apple", 9: "Muskmelon", 10: "Watermelon", 11: "Grapes", 12: "Mango", 13: "Banana",
                 14: "Pomegranate", 15: "Lentil", 16: "Blackgram", 17: "Mungbean", 18: "Mothbeans",
                 19: "Pigeonpeas", 20: "Kidneybeans", 21: "Chickpea", 22: "Coffee"}

predict = recommendation(N,P,k,temperature,humidity,ph,rainfall)

if predict[0] in crop_dict:
    crop = crop_dict[predict[0]]
    print("{} is a best crop to be cultivated ".format(crop))
else:
    print("Sorry are not able to recommend a proper crop for this environment")

Jute is a best crop to be cultivated 


**2.**

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/Colab/crop.csv')

# Prepare the data
X = data.drop('Crop', axis=1).values
y = data['Crop'].astype('category').cat.codes.values  # Convert crop labels to numerical codes

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = y_train.fillna(0)
# Normalize the features (optional but recommended for neural networks)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

nb_train = len(train_y)
nb_test = len(test_y)
nb_input = train_x.shape[1]
nb_classes = len(np.unique(train_y))

print(f'Training samples: {nb_train}, Testing samples: {nb_test}')
print(f'Number of features: {nb_input}, Number of classes: {nb_classes}')

# Parameters
learning_rate = 0.001
nb_epochs = 16
batch_size = 128

# Architecture parameters
nb_hidden1 = 256
nb_hidden2 = 256

# Initialize weights and biases
w = {
    '1': tf.Variable(tf.random.normal([nb_input, nb_hidden1], dtype=tf.float64)),
    '2': tf.Variable(tf.random.normal([nb_hidden1, nb_hidden2], dtype=tf.float64)),
    'out': tf.Variable(tf.random.normal([nb_hidden2, nb_classes], dtype=tf.float64))
}

b = {
    '1': tf.Variable(tf.random.normal([nb_hidden1], dtype=tf.float64)),
    '2': tf.Variable(tf.random.normal([nb_hidden2], dtype=tf.float64)),
    'out': tf.Variable(tf.random.normal([nb_classes], dtype=tf.float64))
}

f = {
    '1': tf.nn.relu,
    '2': tf.nn.relu,
    'out': tf.nn.softmax
}

def runNN(x):
    z1 = tf.add(tf.matmul(x, w['1']), b['1'])
    a1 = f['1'](z1)
    z2 = tf.add(tf.matmul(a1, w['2']), b['2'])
    a2 = f['2'](z2)
    z_out = tf.add(tf.matmul(a2, w['out']), b['out'])
    out = f['out'](z_out)

    pred = tf.argmax(out, 1)

    return pred, z_out

# Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Training
for epoch in range(nb_epochs):
    epoch_loss = 0
    nb_batches = int(nb_train / batch_size)
    for i in range(nb_batches):
        x = train_x[i*batch_size : (i+1)*batch_size, :]
        y = train_y[i*batch_size : (i+1)*batch_size]
        y_onehot = tf.one_hot(y, nb_classes)

        with tf.GradientTape() as tape:
            _, z_out = runNN(x)
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=z_out, labels=y_onehot))

        grads = tape.gradient(loss, [w['1'], w['2'], w['out'], b['1'], b['2'], b['out']])
        opt.apply_gradients(zip(grads, [w['1'], w['2'], w['out'], b['1'], b['2'], b['out']]))

        epoch_loss += loss

    epoch_loss /= nb_batches
    print(f'Epoch: {epoch+1}/{nb_epochs} | Avg loss: {epoch_loss:.5f}')

# Testing
x = test_x
y = test_y

pred, _ = runNN(x)
pred_correct = tf.equal(pred, y)
accuracy = tf.reduce_mean(tf.cast(pred_correct, tf.float32))
print(f'Test acc: {accuracy:.3f}')

Training samples: 1760, Testing samples: 440
Number of features: 7, Number of classes: 22
Epoch: 1/16 | Avg loss: 605.97708
Epoch: 2/16 | Avg loss: 394.41173
Epoch: 3/16 | Avg loss: 245.84624
Epoch: 4/16 | Avg loss: 137.68033
Epoch: 5/16 | Avg loss: 78.60423
Epoch: 6/16 | Avg loss: 42.72681
Epoch: 7/16 | Avg loss: 24.01464
Epoch: 8/16 | Avg loss: 15.28983
Epoch: 9/16 | Avg loss: 10.64890
Epoch: 10/16 | Avg loss: 7.90964
Epoch: 11/16 | Avg loss: 6.08363
Epoch: 12/16 | Avg loss: 4.73579
Epoch: 13/16 | Avg loss: 3.76803
Epoch: 14/16 | Avg loss: 3.10873
Epoch: 15/16 | Avg loss: 2.59800
Epoch: 16/16 | Avg loss: 2.20821
Test acc: 0.864


In [ ]:

crop_mapping = dict(enumerate(data['Crop'].astype('category').cat.categories))

print(crop_mapping)

# Function to recommend the best crop based on new input conditions
def recommend_crop(new_conditions):
    new_conditions_scaled = scaler.transform([new_conditions])
    pred, _ = runNN(new_conditions_scaled)
    recommended_crop = crop_mapping[pred.numpy()[0]]
    return recommended_crop

# Example usage
new_conditions = [90, 42, 43, 20.87974371, 82.00274423, 6.502985292, 202.9355362]
best_crop = recommend_crop(new_conditions)
print("{} is a best crop to be cultivated ".format(best_crop))

{0: 'Apple', 1: 'Banana', 2: 'Blackgram', 3: 'ChickPea', 4: 'Coconut', 5: 'Coffee', 6: 'Cotton', 7: 'Grapes', 8: 'Jute', 9: 'KidneyBeans', 10: 'Lentil', 11: 'Maize', 12: 'Mango', 13: 'MothBeans', 14: 'MungBean', 15: 'Muskmelon', 16: 'Orange', 17: 'Papaya', 18: 'PigeonPeas', 19: 'Pomegranate', 20: 'Rice', 21: 'Watermelon'}
Rice is a best crop to be cultivated 


**3.**

In [ ]:
!pip install tensorflow==2.10 \
gymnasium~=0.29.1 \
minigrid~=2.3.1 \
numpy~=1.26.4 \
pygame~=2.5.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 54.6 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
  

Record

In [ ]:
import gymnasium as gym
import numpy as np
import pygame

from minigrid.core.actions import Actions
from minigrid.wrappers import FullyObsWrapper, ImgObsWrapper


class ManualControl:
    def __init__(self, env):
        self.env = env
        self.closed = False
        self.last_obs = None
        self.observations = []
        self.actions = []

    def start(self):
        self.reset()

        while not self.closed:
            for event in pygame.event.get():
                if event.type == pygame.KEYDOWN:
                    key = pygame.key.name(int(event.key))
                    self.key_handler(key)

        self.env.close()

        if len(self.observations) > 0 and len(self.actions) == len(self.observations):
            np.save('observations.npy', self.observations)
            np.save('actions.npy', self.actions)
        else:
            print("No data saved")

    def step(self, action):
        self.observations.append(self.last_obs)
        self.actions.append(action)

        self.last_obs, _, terminated, truncated, _ = self.env.step(action)

        if terminated or truncated:
            self.reset()
        else:
            self.env.render()

    def reset(self):
        self.last_obs, _ = self.env.reset()
        self.env.render()

    def key_handler(self, key):
        # print(self.observations)
        if key == "escape":
            self.closed = True
            return
        if key == "backspace":
            self.reset()
            return

        key_to_action = {
            "left": Actions.left,
            "right": Actions.right,
            "up": Actions.forward,
            "space": Actions.toggle,
            "tab": Actions.pickup,
        }
        if key in key_to_action.keys():
            action = key_to_action[key]
            self.step(action)


env = gym.make(
    "MiniGrid-Empty-6x6-v0",
    render_mode="human",
    highlight=False,
    screen_size=640
)

env = FullyObsWrapper(env)
env = ImgObsWrapper(env)

manual_control = ManualControl(env)
manual_control.start()


Training

In [ ]:
import tensorflow as tf
import numpy as np

# Učitava zabeležene podatke
observations = np.load('/content/drive/MyDrive/Colab/observations.npy', allow_pickle=True)
actions = np.load('/content/drive/MyDrive/Colab/actions.npy', allow_pickle=True)

# Određuje oblik ulaza i broj mogućih akcija
input_shape = observations.shape[1:]  # Oblik jedne opservacije
num_actions = len(np.unique(actions))  # Broj jedinstvenih akcija

# Definiše model neuronske mreže
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=input_shape),  # Izravnava ulaz
    tf.keras.layers.Dense(128, activation='relu'),     # Skriveni sloj sa 128 neurona
    tf.keras.layers.Dense(64, activation='relu'),      # Skriveni sloj sa 64 neurona
    tf.keras.layers.Dense(num_actions, activation='softmax')  # Izlazni sloj sa brojem neurona jednakim broju akcija
])

# Kompajlira model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Trenira model
model.fit(observations, actions, epochs=100, batch_size=32)

# Čuva trenirani model
model.save('/content/drive/MyDrive/Colab/trained_model.h5')

print("Trening je završen i model je sačuvan.")


Epoch 1/100
9/9 [==============================] - 1s 3ms/step - loss: 1.5056 - accuracy: 0.4667
Epoch 2/100
9/9 [==============================] - 0s 2ms/step - loss: 0.9721 - accuracy: 0.6593
Epoch 3/100
9/9 [==============================] - 0s 2ms/step - loss: 0.8067 - accuracy: 0.6889
Epoch 4/100
9/9 [==============================] - 0s 2ms/step - loss: 0.7425 - accuracy: 0.6889
Epoch 5/100
9/9 [==============================] - 0s 2ms/step - loss: 0.7340 - accuracy: 0.6889
Epoch 6/100
9/9 [==============================] - 0s 2ms/step - loss: 0.6992 - accuracy: 0.7074
Epoch 7/100
9/9 [==============================] - 0s 2ms/step - loss: 0.6955 - accuracy: 0.6889
Epoch 8/100
9/9 [==============================] - 0s 2ms/step - loss: 0.6833 - accuracy: 0.6889
Epoch 9/100
9/9 [==============================] - 0s 2ms/step - loss: 0.6685 - accuracy: 0.7296
Epoch 10/100
9/9 [==============================] - 0s 2ms/step - loss: 0.6752 - accuracy: 0.7111
Epoch 11/100
9/9 [===========

Test

In [ ]:
import tensorflow as tf
import numpy as np
import gymnasium as gym

from minigrid.wrappers import FullyObsWrapper, ImgObsWrapper

LOCAL = False

# Load the trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/Colab/trained_model.h5')

env = gym.make(
    "MiniGrid-Empty-Random-6x6-v0",
    render_mode="human" if LOCAL else "rgb_array",
    highlight=False,
    screen_size=640
)

env = FullyObsWrapper(env)
env = ImgObsWrapper(env)

rewards = []

for episode in range(10):
    obs, _ = env.reset()
    step = 0
    terminated = False
    truncated = False

    while not terminated and not truncated and step < 30:
        if LOCAL:
            env.render()
        action = model.predict(obs.reshape(-1, *obs.shape), verbose=0)
        obs, reward, terminated, truncated, _ = env.step(np.argmax(action))
        step += 1
        # print("step")

    print(f"{episode=} {reward=}")
    rewards.append(reward)

env.close()
print(f"mean reward: {np.mean(rewards)}")



pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.width to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.width` for environment variables or `env.get_wrapper_attr('width')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.height to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.height` for environment variables or `env.get_wrapper_attr('height')` that will search the reminding wrappers.
  logger.warn(


episode=0 reward=0.96875
episode=1 reward=0.975
episode=2 reward=0.9625
episode=3 reward=0.98125
episode=4 reward=0.98125
episode=5 reward=0.96875
episode=6 reward=0.95625
episode=7 reward=0
episode=8 reward=0.975
episode=9 reward=0.96875
mean reward: 0.87375
